In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
# Relative path to data files
DATA_DIR = '../data/'

In [3]:
# List data files in DATA_DIR
files = sorted(
    [file for file in os.listdir(DATA_DIR) if file[-4:] == '.csv'])
files

['elcampoTX_2007_2015.csv', 'eugeneOR_2007_2015.csv']

In [4]:
# Read data
data_daily = pd.read_csv(DATA_DIR + files[0])

# Convert DATE to datetime (instead of string)
data_daily['DATE']= pd.to_datetime(data_daily['DATE'])
data_daily.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,TMAX,TMIN,TOBS
0,USC00412266,"DANEVANG 1 W, TX US",29.0567,-96.2319,21.3,2007-01-01,0.00,63.0,37.0,37.0
1,USC00412266,"DANEVANG 1 W, TX US",29.0567,-96.2319,21.3,2007-01-02,0.00,55.0,36.0,37.0
2,USC00412266,"DANEVANG 1 W, TX US",29.0567,-96.2319,21.3,2007-01-03,0.00,55.0,37.0,47.0
3,USC00412266,"DANEVANG 1 W, TX US",29.0567,-96.2319,21.3,2007-01-04,1.48,54.0,47.0,49.0
4,USC00412266,"DANEVANG 1 W, TX US",29.0567,-96.2319,21.3,2007-01-05,0.05,57.0,46.0,55.0


In [5]:
# Extract YEAR and MONTH from DATE
data_daily['YEAR'] = data_daily.DATE.apply(lambda x: x.year)
data_daily['MONTH'] = data_daily.DATE.apply(lambda x: x.month)
data_daily.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,TMAX,TMIN,TOBS,YEAR,MONTH
0,USC00412266,"DANEVANG 1 W, TX US",29.0567,-96.2319,21.3,2007-01-01,0.00,63.0,37.0,37.0,2007,1
1,USC00412266,"DANEVANG 1 W, TX US",29.0567,-96.2319,21.3,2007-01-02,0.00,55.0,36.0,37.0,2007,1
2,USC00412266,"DANEVANG 1 W, TX US",29.0567,-96.2319,21.3,2007-01-03,0.00,55.0,37.0,47.0,2007,1
3,USC00412266,"DANEVANG 1 W, TX US",29.0567,-96.2319,21.3,2007-01-04,1.48,54.0,47.0,49.0,2007,1
4,USC00412266,"DANEVANG 1 W, TX US",29.0567,-96.2319,21.3,2007-01-05,0.05,57.0,46.0,55.0,2007,1


In [6]:
# Aggregate by month: get total monthly precipitation, and monthly 
# avg, min, and max temps
data_monthly = data_daily.groupby(['YEAR', 'MONTH'])\
    .agg({'PRCP': sum, 'TOBS': np.mean, 'TMAX': max, 'TMIN': min})
data_monthly.head()

PRCP       TOBS  TMAX  TMIN
YEAR MONTH                              
2007 1      28.20  45.565789  78.0  26.0
     2       1.57  50.666667  84.0  23.0
     3      17.97  61.171053  91.0  30.0
     4      17.50  62.197368  89.0  36.0
     5      18.86  69.951220  96.0  56.0

Combine the above into a function:

In [7]:
def aggregate_daily_file_to_monthly(filepath):
    data_daily = pd.read_csv(filepath)
    if 'TOBS' not in list(data_daily):
        try:
            data_daily = data_daily.rename(columns={'TAVG': 'TOBS'})
        except KeyError:
            print('Neither "TOBS" nor "TAVG" found in data set.')
            return None
    data_daily['DATE']= pd.to_datetime(data_daily['DATE'])
    data_daily['YEAR'] = data_daily.DATE.apply(lambda x: x.year)
    data_daily['MONTH'] = data_daily.DATE.apply(lambda x: x.month)
    data_monthly = data_daily.groupby(['YEAR', 'MONTH'])\
        .agg({'YEAR': min, 
              'MONTH': min,
              'PRCP': sum, 
              'TOBS': np.mean, 
              'TMAX': max, 
              'TMIN': min})
    return data_monthly.reset_index(drop=True)

In [8]:
files

['elcampoTX_2007_2015.csv', 'eugeneOR_2007_2015.csv']

In [9]:
aggregate_files = []
for file in files:
    print('Aggregating data for %s...\t\t' % file)
    aggregate_files.append(
        aggregate_daily_file_to_monthly(DATA_DIR + file))

Aggregating data for elcampoTX_2007_2015.csv...		
Aggregating data for eugeneOR_2007_2015.csv...		


In [10]:
print(files[0].split('_')[0])
print(aggregate_files[0].shape)
aggregate_files[0].head()

elcampoTX
(108, 6)


,YEAR,MONTH,PRCP,TOBS,TMAX,TMIN
0,2007,1,28.20,45.565789,78.0,26.0
1,2007,2,1.57,50.666667,84.0,23.0
2,2007,3,17.97,61.171053,91.0,30.0
3,2007,4,17.50,62.197368,89.0,36.0
4,2007,5,18.86,69.951220,96.0,56.0


In [11]:
print(files[1].split('_')[0])
print(aggregate_files[1].shape)
aggregate_files[1].head()

eugeneOR
(108, 6)


,YEAR,MONTH,PRCP,TOBS,TMAX,TMIN
0,2007,1,4.88,NaN,59.0,10.0
1,2007,2,7.37,NaN,68.0,14.0
2,2007,3,3.00,NaN,76.0,22.0
3,2007,4,2.99,NaN,79.0,24.0
4,2007,5,2.67,NaN,88.0,27.0


In [12]:
# Merge into a single data set
merged = aggregate_files.pop()

In [13]:
while len(aggregate_files):
    next_file = aggregate_files.pop()
    merged = merged.append(next_file)

In [14]:
print(merged.shape)
merged.head()

(216, 6)


,YEAR,MONTH,PRCP,TOBS,TMAX,TMIN
0,2007,1,4.88,NaN,59.0,10.0
1,2007,2,7.37,NaN,68.0,14.0
2,2007,3,3.00,NaN,76.0,22.0
3,2007,4,2.99,NaN,79.0,24.0
4,2007,5,2.67,NaN,88.0,27.0


In [16]:
# Write combined data back to .csv
merged.to_csv(DATA_DIR + 'merged_%d_sites.csv' % len(files), index=False)

In [17]:
!ls $DATA_DIR

elcampoTX_2007_2015.csv eugeneOR_2007_2015.csv  merged_2_sites.csv
